In [1]:
from spacebase import *
from src.api import *
import pandas as pd
import datetime
import pickle

In [2]:
FILENAME = "spacedata.pkl"

In [3]:
def load_database():
    return pd.read_pickle(FILENAME)

In [4]:
def save_database(database):
    return pickle.dump(database, open(FILENAME, 'wb'))

In [5]:
database = load_database()

In [181]:
save_database(database)

In [82]:
database['markets'] = database['markets'].reset_index()

In [83]:
database['markets'] = database['markets'].set_index(['location', 'date'])

In [94]:
max(database['markets'].loc['OE-PM-TR'].index)

'2022-07-17'

In [79]:
datestr

'2022-07-17'

In [95]:
database['markets'].loc['OE-PM-TR'].loc[max(database['markets'].loc['OE-PM-TR'].index)]

,symbol,volumePerUnit,pricePerUnit,spread,purchasePricePerUnit,sellPricePerUnit,quantityAvailable
date,,,,,,,
2022-07-17,METALS,1,2,1,3,1,93751
2022-07-17,CONSTRUCTION_MATERIALS,1,140,4,144,136,0
2022-07-17,FUEL,1,2,1,3,1,87271
2022-07-17,FUSION_REACTORS,6,20400,196,20596,20204,0


In [51]:
database['user']

{'username': 'crushbone',
 'shipCount': 2,
 'structureCount': 0,
 'joinedAt': '2022-07-03T20:54:08.115Z',
 'credits': 685213}

In [98]:
database['myShips'].loc['cl55thau617774315s68iz3fpjj']

location                                                      OE-NY
x                                                                43
y                                                               -46
cargo             [{'good': 'FUEL', 'quantity': 14, 'totalVolume...
spaceAvailable                                                   36
type                                                        JW-MK-I
class                                                          MK-I
maxCargo                                                         50
loadingSpeed                                                     25
speed                                                             1
manufacturer                                               Jackshaw
plating                                                           5
weapons                                                           5
Name: cl55thau617774315s68iz3fpjj, dtype: object

In [164]:
database['myShips'].loc['cl55thau617774315s68iz3fpjj'].cargo

[{'good': 'FUEL', 'quantity': 14, 'totalVolume': 14}]

In [180]:
for ship in database['myShips'].iterrows():
    cargo = {}
    for item in ship[1].cargo:
        cargo[item['good']] = {'quantity': item['quantity'], 'totalVolume': item['totalVolume']}
    database['myShips'].loc[ship[1].name].cargo = cargo

C:\Users\kayla\AppData\Local\Temp\ipykernel_16564\2394785272.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  database['myShips'].loc[ship[1].name].cargo = cargo


In [174]:
database['myShips'].cargo.cl55thau617774315s68iz3fpjj

[{'good': 'FUEL', 'quantity': 14, 'totalVolume': 14}]

In [135]:
sample_ship = pd.DataFrame(get_ships()[0]).set_index('id')

In [152]:
sample_cargo = sample_ship.cargo.cl55thau617774315s68iz3fpjj

In [160]:
pd.DataFrame(sample_cargo, index=[0]).set_index('good')

,quantity,totalVolume
good,,
FUEL,14,14


In [126]:
sample_cargo = database['myShips'].loc['cl55thau617774315s68iz3fpjj'].cargo

In [13]:
# PLAYER INFO
def get_my_status():
  return database['user']
  
def get_money():
  return get_my_status()['credits']

# FLIGHTPLAN
def get_flightplan(flightPlan_id):
  return database['flightPlans'].loc(flightPlan_id)
  return make_request(f'{flightplan_url}/{flightPlan_id}', params=params).json()['flightPlan']
  
def make_flightplan(ship_id, destination):
  new_data = make_request(f'{flightplan_url}', params={'shipId':ship_id, 'destination':destination}, type="POST").json()['flightPlan']
  database['flightPlans'] = pd.concat([database['flightPlans'], new_data], ignore_index = True)
  return new_data

# LOANS
def get_loans():
  return database['loans']
  return make_request(url= f'{loans_url}').json()['loans']

def pay_loan(loan_id):
  return make_request(f'{loans_url}/{loan_id}', type="PUT").json()

def take_loan(loan_type):
  return make_request(f'{loans_url}', params={'type':loan_type}, type="POST").json()

# LOCATIONS
def get_location_data(location_id, params={}):
  return database['locations'].query(f"symbol == {location_id}")
  return make_request(f'{location_url}/{location_id}', params=params).json()['location']

def get_market_data(location_id, params={}):
  return database['markets'].loc[location_id].loc[max(database['markets'].loc[location_id].index)]
  return make_request(f'{location_url}/{location_id}/marketplace', params=params).json()['marketplace']

def get_ships(location_id, params={}):
  return database['ships'].query(f"location == {location_id}")
  return make_request(f'{location_url}/{location_id}/ships', params=params).json()['ships']

def get_structures(location_id, params={}): # TODO
  return make_request(f'{location_url}/{location_id}/structures', params=params).json()['structures']

# COMMERCE
def purchace_goods(ship_id, good, quantity):
  new_data = make_request(f'{purchace_url}', params={'shipId':ship_id,'quantity':quantity,'good':good}, type='POST').json()
  database['myShips'].query(f"id == '{ship_id}'").update(new_data['ship'])
  database['user']['credits'] = new_data['credits']
  return new_data

def sell_goods(ship_id, good, quantity):
  new_data = make_request(f'{sell_url}', params={'shipId':ship_id,'quantity':quantity,'good':good}, type='POST').json()
  database['myShips'].query(f"id == '{ship_id}'").update(new_data['ship'])
  database['user']['credits'] = new_data['credits']
  return new_data

def fuel_ship(ship_id,fuel_amount): #
  return purchace_goods(f'{ship_id}', 'FUEL', fuel_amount).json()

# SHIPS
def buy_ship(location, type):
  new_data = make_request(f'{myShips_url}',params = {'location': location, 'type': type}, type = "POST").json()
  database['myShips'] =  pd.concat([database['myShips'], new_data['ship']], ignore_index = True)
  database['user']['credits'] = new_data['credits']
  return new_data

def get_ships(params={}):
  return database['myShips']

def get_ship_data(ship_id, params={}):
  return database['myShips'].loc[ship_id]
  return make_request(f'{myShips_url}/{ship_id}', params=params).json()['ship']

def jettison_ship_cargo(ship_id, good, quantity):
  return make_request(f'{myShips_url}/{ship_id}/jettison', params = {'shipId':ship_id, 'good':good, 'quantity':quantity}, type="POST").json()

def scrap_ship(ship_id):
  return make_request(f'{myShips_url}/{ship_id}', type = "DELETE").json()

def transfer_ship_cargo(from_ship_id, to_ship_id, good, quantity):
  return make_request(f'{myShips_url}/{from_ship_id}', params = {'toShipId':to_ship_id, 'good': good,'quantity':quantity}, type ="POST").json()

#STRUCTURES
def get_structure_data(structure_id):
  return make_request(f'{struct_url}/{structure_id}')['structure']

def get_my_structure_data(structure_id):
  return make_request(f'{my_struct_url}/{structure_id}')['structure']

def get_my_structures():
  return make_request(f'{my_struct_url}')['structures']

def create_structure(location_id, structure_type):
  return make_request(f'{my_struct_url}', params = {'location': location_id, 'type': structure_type}).json()['structure']

def deposit_goods_to_owned_structure(structure_id, ship_id, good, quantity):
  return make_request(f'{my_struct_url}/deposit', params = {'structureId': structure_id, 'shipId': ship_id, 'good': good, 'quantity': quantity}).json()

def deposit_goods_to_public_structure(structure_id, ship_id, good, quantity):
  return make_request(f'{struct_url}/{structure_id}/deposit', params = {'structureId': structure_id, 'shipId': ship_id, 'good': good, 'quantity': quantity}).json()

def transfer_goods_from_owned_structure(structure_id, ship_id, good, quantity):
  return make_request(f'{my_struct_url}/transfer', params = {'structureId': structure_id, 'shipId': ship_id, 'good': good, 'quantity': quantity}).json()

#SYSTEMS
def get_system_ship_listings(system_id, params={}):
  return make_request(f'{system_url}/{system_id}/ship-listings', params=params).json()['shipListings']

def get_system_flightplans(system_id, params ={}):
  return make_request(f'{system_url}/{system_id}/flight-plans', params=params).json()['flightPlans']

def get_system_ships(system_id, params={}):
  return make_request(f'{system_url}/{system_id}/ships', params=params).json()['ships']

def get_system_locations(system_id, params={}):
  return make_request(f'{system_url}/{system_id}/locations', params=params).json()['locations']

def get_system_data(system_id, params={}):
  return make_request(f'{system_url}/{system_id}', params = params).json()['system']

#TYPES
def get_available_goods(params={}):
  return make_request(f'{types_url}/goods', params=params).json()['goods']

def get_available_loans(params={}):
  return make_request(f'{types_url}/loans').json()['loans']

def get_available_structures(params={}):
  return make_request(f'{types_url}/structures').json()['structures']

def get_available_ships(params={}):
  return make_request(f'{types_url}/ships').json()['ships']

#USERS
def claim_username(username):
  response = make_request(f'https://api.spacetraders.io/users/{username}/claim').json()
  [print(key,':',value) for key, value in response.items()]
  return print()

def warp_jump(ship_id):
  return make_request(f'https://api.spacetraders.io/my/warp-jumps', params={'shipId':ship_id}).json()['flightPlan']


In [20]:
def initialize_database(): ## TODO THIS SUCKS
    return {"locations":pandas.DataFrame({}),
    "goods":pandas.DataFrame({}),
    "loans":pandas.DataFrame({}),
    "structures":pandas.DataFrame({}),
    "ships":pandas.DataFrame({}),
    "user":{},"flightPlans":pandas.DataFrame({}),
    "markets":pandas.DataFrame({}),
    "myShips":pandas.DataFrame({}),
    "myStructures":pandas.DataFrame({})}

In [43]:
database['locations'].query("symbol == 'OE-PM'")

,symbol,type,name,x,y,allowsConstruction,traits,messages
0,OE-PM,PLANET,Prime,-19,3,False,"[METAL_ORES, SOME_NATURAL_CHEMICALS]",NaN


In [51]:
def update_markets(location):
    if not ships_present(location): print(f"ERROR: NO SHIPS PRESENT AT {location}")
    else:
        data = get_market_data(location)
        timestamp = str(datetime.datetime.today())
        for datum in data:
            datum['location'] = location
            datum['date'] = pd.to_datetime(timestamp)

        database['markets'] = pd.concat([database['markets'], pd.DataFrame(data)])
        

In [9]:
database['markets']

,symbol,volumePerUnit,pricePerUnit,spread,purchasePricePerUnit,sellPricePerUnit,quantityAvailable,location,date
0,METALS,1,2,1,3,1,93751,OE-PM-TR,2022-07-17
1,CONSTRUCTION_MATERIALS,1,140,4,144,136,0,OE-PM-TR,2022-07-17
2,FUEL,1,2,1,3,1,87271,OE-PM-TR,2022-07-17
3,FUSION_REACTORS,6,20400,196,20596,20204,0,OE-PM-TR,2022-07-17
0,METALS,1,2,1,3,1,93591,OE-NY,2022-07-17
1,NARCOTICS,1,115,5,120,110,14,OE-NY,2022-07-17
2,FUEL,1,32,1,33,31,22625,OE-NY,2022-07-17
3,EXPLOSIVES,1,92,2,94,90,10,OE-NY,2022-07-17
4,FOOD,1,35,1,36,34,11,OE-NY,2022-07-17
5,RARE_METALS,1,48,4,52,44,47359,OE-NY,2022-07-17
